# Agregace a spojování

Stáhneme si soubory, se kterými budeme pracovat.

In [ ]:
import requests

file_names = ["u202.csv", "u203.csv", "u302.csv", "studenti.csv", "predsedajici.csv"]

for file in file_names:
    url = "https://kodim.cz/cms/assets/kurzy/python-data-1/python-pro-data-1/agregace-a-spojovani/{}".format( file )
    r = requests.get(url)
    open(file, "wb").write(r.content)
    print(f"File {file} downloaded.")

Vytvoříme pandas DataFrame pro soubory s výsledky maturitní zkoušky.

In [ ]:
import pandas

u202 = pandas.read_csv("u202.csv")
u203 = pandas.read_csv("u203.csv")
u302 = pandas.read_csv("u302.csv")

Podíváme se na data.

In [ ]:
u202

## Práce s chybějícími hodnotami
anglicky _missing values_


**Co je chybějící hodnota?**
- hodnota `NaN` (Not a Number)
    - _napr. prazdna bunka v csv --> rovnou nahrazena NaN_
- `None` = zabudovaná konstanta v Pythonu 
- `pd.NA` = zatím experiment hodnota, která by měla reprezentovat chybějící hodnoty všech typů

<br>
<br>

**Co není chybějící hodnota?**
- řetězec `''` nebo `'NaN'` and `'None'`
- `inf` / `-inf` = infinity je nedefinované číslo, kladné nebo záporné

<br>
<br>

**Jak s chybějící hodnotou zacházet?**
1. Nejlepší je vždy ověření, proč údaje chybí (např. u poskytovatele dat) a pokud je to možné, zajistit jejich doplnění.
2. Nahradit chybějící hodnoty jinými hodnotami.
3. Odstranit všechny řádky s chybějícími daty z datového setu.
4. Vyčlenit je do separátního datasetu a zpracovat je zvlášť.

_Důležité je mít na paměti, že vyřazením některých řádků může dojít ke zkreslení výsledků analýzy!_

### Odstranění neúplných řádků

**Jak chybějící hodnoty najít a odstranit**



**`isnull()`** 

**`notnull()`**

Obě metody vrací hodnoty `True` nebo `False`

Vyzkoušíme si `isnull()`.

In [ ]:
u202['znamka'].isnull()

## Alternativa 
# u202['znamka'].isna() # isna() se NIJAK neliší od isnull() 

Vyzkoušíme si `notnull()`.

In [ ]:
u202['znamka'].notnull()

## Alternativa
# u202['znamka'].notna()

Vybereme pomocí `isnull()` jen řádky, které mají chybějící hodnoty.

In [ ]:
## Najedeme je ty, kde isnull() je True
jen_null_radky = u202[u202["znamka"].isnull()]
jen_null_radky

### Další metody na práci s chybějícími hodnotami


**`dropna()`**
- vrátí datový set očištěn od chybějících dat
- `dropna(axis=0)` --> odstraní všechny řádky, které obsahují chybějící data
- `dropna(axis=1)` --> odstraní všechny sloupce, které obsahují chybějící data
- Dokumentace: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dropna.html
<br>
<br>


**`fillna(x)`**
- nahradí všechna chybějící data a hodnoty hodnotou x
- Dokumentace: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.fillna.html


In [ ]:
## Příklady těchto metod - pojďme si to vyzkoušet
testovaci_data = u202[u202['znamka'].isnull()]
testovaci_data

In [ ]:
# Odstranění řádků s dropna()
testovaci_data.dropna()

## Alternativa
# testovaci_data.dropna(axis=0)

In [ ]:
# Odstranění sloupců s dropna()
testovaci_data.dropna(axis=1)

In [ ]:
# Nahrazení chybějících hodnot
testovaci_data.fillna('Hodnota chybí')

## Spojení dat

Pojďme spojit naše tři tabulky dohromady.


- Tabulky spojíme pod sebe. 
- Budeme stále mít tři sloupce.
- Počet řádků bude odpovídat počtu řádků všech tří tabulek dohromady.
- _V SQL tomu odpovídá operace `UNION`._

Použijeme naše DataFrames a očistíme je o řádky s chybějícími hodnotami

In [ ]:
## Tak to pojďme vyčistit
u202 = pandas.read_csv('u202.csv').dropna()
u203 = pandas.read_csv('u203.csv').dropna()
u302 = pandas.read_csv('u302.csv').dropna()

**Funkce `concat`**


! Pozor: funkce rozbije index. 
<br>
<br>
Spojí totiž indexy tabulek za sebe např. 1 2 3 1 2 3 1 2 3. <br>
Pomůže nám parameter `ignore_index`

In [ ]:
maturita = pandas.concat([u202, u203, u302])
maturita

In [ ]:
maturita = pandas.concat([u202, u203, u302], ignore_index=True)
maturita

**Vytvoření nového sloupce**

Spojením tabulek jsme ztratili informace o čísle místnosti. 

Můžeme si ale číslo místnosti před spojením uložit do nového sloupečku.

In [ ]:
u202['mistnost'] = 'u202'
u203['mistnost'] = 'u203'
u302['mistnost'] = 'u302'

maturita = pandas.concat([u202, u203, u302], ignore_index=True)

**Uložení tabulky do souboru**

In [ ]:
# dataframe.to_csv('soubor.csv', index=False)

maturita.to_csv('maturita.csv', index=False)

Finální tabulku, kterou jsme právě vytvořili, si můžete pro kontrolu stáhnout zde: 

<https://stage.kodim.cz/cms/assets/kurzy/python-data-1/python-pro-data-1/agregace-a-spojovani/maturita.csv>

## Propojení dat

- Tabulky můžeme spojit více způsoby. 
- _V SQL tomu odpovídá operace `JOIN`._
- Výsledná tabulka bude mít více sloupců.
- Počet řádků na typu propojení.


**Typy propojení**

![Druhy propojeni](assets/type_of_join_operations.png)

**Přidáme další tabulku**

Naše výsledky byly anonymní. Pokud bychom ale chtěli vytisknout maturitní vysvědčení, potřebujeme k číslům studenta zjistit jejich jména. 

Jména najdeme v samostatné tabulce `studenti.csv`. Načtěme si jej jako `DataFrame`.

In [ ]:
studenti = pandas.read_csv('studenti.csv')

U operace `JOIN` jsou důležité dvě věci:

- Podle jakého sloupce (nebo jakých sloupců) dvě různé tabulky propojujeme.
- Co udělat v případě, že pro nějaké řádky nemám ve druhé tabulce odpovídající hodnotu.

**Funkce `merge`**

Dokumentace: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html

In [ ]:
propojeny_df = pandas.merge(u202, studenti)

Ve výchozím nastavení funkce merge() ponechá pouze řádky, které mají záznamy v obou tabulkách. 

V SQL bychom tuto operaci označili jako INNER JOIN.

In [ ]:
"""
Pokud by například nějaký student nebyl uvedený v tabulce se studenty, jeho maturitní výsledek by zmizel. 
U nového DataFrame bychom tedy měli zkontrolovat, zda má propojeny_df stejný počet řádků jako u202.
"""

print(u202.shape)

print(propojeny_df.shape)

**Tabulka s předsedy maturitních komisí**

In [ ]:
preds = pandas.read_csv('predsedajici.csv')

In [ ]:
novy_propojeny_df = pandas.merge(propojeny_df, preds)
novy_propojeny_df.head()

Tentokrát jsme příliš neuspěli, výsledný DataFrame je prázdný. 

Protože v obou DataFrame máme sloupec **jmeno**, v jednom případě však jde o jméno studenta a ve druhém o jméno předsedy komise. 

Musíme říct, že chceme data spojit pouze podle sloupce den.

In [ ]:
novy_propojeny_df = pandas.merge(propojeny_df, preds, on=['den'])
novy_propojeny_df

**Zmizely nám řádky!**

To znamená, že funkce `merge()` **nenašla pro všechna zkoušení odpovídajícího předsedu**. 

<br>

Zkusme nyní říct funkci `merge()`, aby nám zachovala v prvním DataFrame ty řádky, pro které nenajde odpovídající záznam. 

Této operaci se v jazyce SQL říká **LEFT OUTER JOIN**. 

My ho provede tak, že funkci `merge()` jako parametr `how` zadáme hodnotu `left`.

In [ ]:
novy_propojeny_df = pandas.merge(propojeny_df, preds, on=['den'], how="left") # how = "outer"
novy_propojeny_df

Zkusme si zobrazit ty řádky, které se nepodařilo propojit. 

Poznáme je tak, že mají prázdný sloupec datum.

Z nějakého důvodu nám nefunguje propojení v případě, že ve sloupci den je hodnota po. 

In [ ]:
novy_propojeny_df[novy_propojeny_df["datum"].isnull()]

**funkce `strip()`**

z řetězce odstraní mezery (a další bílé znaky) na začátku a na konci

In [ ]:
preds["den"] = preds["den"].str.strip()


In [ ]:
novy_propojeny_df = pandas.merge(propojeny_df, preds, on=['den'], how="left")
novy_propojeny_df

**Přejmenování sloupců**

In [ ]:
novy_propojeny_df = novy_propojeny_df.rename(columns={'jmeno_x': 'jmeno', 'jmeno_y': 'predseda'})

## Agregace

**metoda `groupby()`**

Pomocí metody groupby vyrobíme z `DataFrame` speciální objekt `DataFrameGroupBy`. 

`maturita.groupby('mistnost')`

**Agregační funkce**

In [ ]:
maturita.groupby('mistnost').count()

Další užitečné agregační funkce jsou například:

* `sum` - součet hodnot,
* `max` - maximální hodnota,
* `min` - minimální hodnota,
* `first` - první hodnota,
* `last` - poslední hodnota,
* `mean` - průměr z hodnot,
* `median` - medián z hodnot.

![Princip agragace](assets/groupby.png)

**Agregace vybraných sloupců**

In [ ]:
maturita.groupby('predmet')['znamka'].mean()

In [ ]:
## Nebo více sloupců
maturita.groupby('predmet')[['znamka', 'cisloStudenta']].mean()

**ČTENÍ NA DOMA**

**Funkce `.agg()`**

Dokumentace: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.agg.html


In [ ]:
maturita.groupby("cisloStudenta").agg({"znamka": ["max", "mean"]})

## Řazení

**Metoda `sort_values`**

Metoda `sort_values` standardně řadí vzestupně. 

Chceme-li řadit sestupně, zadáme jí parametr `ascending` a nastavíme ho na `False`.

In [ ]:
## Řazení hodnot v Series
## https://pandas.pydata.org/docs/reference/api/pandas.Series.sort_values.html

vysledky_predmetu = maturita.groupby('predmet')['znamka'].max() 
vysledky_predmetu.sort_values(ascending=False)

In [ ]:
## Řazení hodnot v DataFramu
## https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sort_values.html

maturita.sort_values(by="znamka")

In [ ]:
## Řazení hodnot v DataFramu s Multiindexem

vysledky_studenty = maturita.groupby("cisloStudenta").agg({"znamka": ["mean"]})
vysledky_studenty.sort_values([("znamka", "mean")])